# **B. Data Analysis Questions**

**Things I learned:**

- multiple common table expression(cte)s
- LEAD() <span style="background-color: rgb(255, 255, 255); color: rgb(77, 81, 86); font-family: arial, sans-serif;">analytic function</span>
- <span style="background-color: rgb(255, 255, 255); color: rgb(77, 81, 86); font-family: arial, sans-serif;">CASE statement<br></span>

### **1\. How many customers has Foodie-Fi ever had?**

In [1]:
SELECT 
	COUNT(DISTINCT(s.customer_id)) as no_of_customers
FROM subscriptions s
JOIN plans p
	ON s.plan_id = p.plan_id;

(1 row(s) affected)

Total execution time: 00:00:00.011

no_of_customers
1000


### <span style="font-size: 14px;"><b>2. What is the monthly distribution of trial plan start_date values for our dataset - use the start of the month as the group by value</b></span>

<span style="font-size: 14px;">-- where plan name is trial</span>

\-- extract month from start date

<span style="font-size: 14px;">-- group by extracted month</span>

In [2]:
SELECT 
	EXTRACT(MONTH FROM s.start_date) as month,
	COUNT(*) as monthly_distribution
FROM subscriptions s
JOIN plans p
	ON s.plan_id = p.plan_id
	WHERE p.plan_name='trial'
	GROUP BY month
	ORDER BY month;

(12 row(s) affected)

Total execution time: 00:00:00.006

month,monthly_distribution
1.0,88
2.0,68
3.0,94
4.0,81
5.0,88
6.0,79
7.0,89
8.0,88
9.0,87
10.0,79


**3. What plan start\_date values occur after the year 2020 for our dataset? Show the breakdown by count of events for each plan\_name**

<span style="font-size: 14px;">-- where start_date &gt; 2020</span>

<span style="font-size: 14px;">-- distinct plan names</span>

<span style="font-size: 14px;">-- count plans</span>

<span style="font-size: 14px;">-- group by distinct plan names</span>

In [3]:
SELECT
	DISTINCT(p.plan_name) as plans,
	COUNT(*)
FROM subscriptions s
JOIN plans p
	ON s.plan_id = p.plan_id
	WHERE EXTRACT(YEAR FROM s.start_date) > 2020
	GROUP BY plans;

(4 row(s) affected)

Total execution time: 00:00:00.004

plans,count
churn,71
pro monthly,60
basic monthly,8
pro annual,63


### **4.What is the customer count and percentage of customers who have churned rounded to 1 decimal place?**

In [4]:
SELECT 
	COUNT(DISTINCT(s.customer_id)) as customer_count,
	ROUND((COUNT(DISTINCT(s.customer_id))::numeric/1000::numeric)*100,1) as churn_customers_percentage
FROM subscriptions s
JOIN plans p
	ON s.plan_id = p.plan_id
	WHERE p.plan_name='churn'

(1 row(s) affected)

Total execution time: 00:00:00.005

customer_count,churn_customers_percentage
307,30.7


In [5]:
with cte as(
	SELECT 
		COUNT(DISTINCT(s.customer_id)) as total_customer_count,
		COUNT(CASE
				WHEN p.plan_name='churn' THEN 1
			  END) as churned_customer_count
	FROM subscriptions s
	JOIN plans p
		ON s.plan_id = p.plan_id)
	SELECT
		*,
		ROUND((churned_customer_count::numeric/total_customer_count::numeric)*100,1) as churn_customers_percentage
	FROM cte;

(1 row(s) affected)

Total execution time: 00:00:00.003

total_customer_count,churned_customer_count,churn_customers_percentage
1000,307,30.7


### <span style="font-size: 14px;"><b>5. How many customers have churned straight after their initial free trial - what percentage is this rounded to the nearest whole number?</b></span>

<span style="font-size: 14px;">-- use lead() to get next immediate plan</span>

<span style="font-size: 14px;">-- use where clause plan_name='trial' and next_plan='churn'</span>

<span style="font-size: 14px;">-- calculate count and percentages</span>

In [6]:
with cte as(
	SELECT 
		s.customer_id,
		s.start_date,
		p.plan_name,
		LEAD(p.plan_name,1) OVER(partition by s.customer_id order by s.start_date) as next_plan
	FROM subscriptions s
	JOIN plans p
		ON s.plan_id = p.plan_id)
		SELECT
			COUNT(*) no_of_customers_churned,
			ROUND((COUNT(*)::numeric/1000::numeric)*100) as churned_customers_percentage
		FROM cte
		WHERE plan_name='trial' and next_plan='churn';

(1 row(s) affected)

Total execution time: 00:00:00.007

no_of_customers_churned,churned_customers_percentage
92,9


### <span style="font-size: 14px;"><b>6. What is the number and percentage of customer plans after their initial free trial?</b></span>

<span style="font-size: 14px;">-- use LEAD() to get next immediate plan</span>

<span style="font-size: 14px;">-- count total customers for each plan</span>

<span style="font-size: 14px;">-- count customers for each plan after their initial free trial</span>

<span style="font-size: 14px;">-- calculate percentage by dividing&nbsp;</span>

In [7]:
with cte2 as(
	with cte1 as(
		SELECT 
			s.customer_id,
			s.start_date,
			p.plan_name,
			LEAD(p.plan_name,1) OVER(partition by s.customer_id order by s.start_date) as next_plan
		FROM subscriptions s
		JOIN plans p
			ON s.plan_id = p.plan_id)
	SELECT
		next_plan,
		COUNT(*) as total_customer_count,
		COUNT(
			CASE
				WHEN plan_name='trial' THEN 1 
			END) as customer_plans_after_free_trial
	FROM cte1
	GROUP BY next_plan)
SELECT
	*,
	ROUND(((customer_plans_after_free_trial::numeric/total_customer_count::numeric)*100 ),2)as percentage
FROM cte2

(5 row(s) affected)

Total execution time: 00:00:00.008

next_plan,total_customer_count,customer_plans_after_free_trial,percentage
NULL,1000,0,0.00
pro annual,258,37,14.34
churn,307,92,29.97
pro monthly,539,325,60.30
basic monthly,546,546,100.00


### <span style="font-size: 14px;"><b>7. What is the customer count and percentage breakdown of all 5 plan_name values at 2020-12-31?</b></span>

<span style="font-size: 14px;">-- count total customers for each plan</span>

<span style="font-size: 14px;">-- count customers for each plan before 2020-12-31 using case statement</span>

<span style="font-size: 14px;">-- calculate percentage&nbsp;</span>

In [8]:
with cte as(
	SELECT 
		p.plan_name,
		COUNT(customer_id) as total_customer_count,
		COUNT(
			CASE
				WHEN EXTRACT(YEAR FROM s.start_date) =2020 THEN 1
			END) as customer_count_in_2020	
	FROM subscriptions s
	JOIN plans p
		ON s.plan_id = p.plan_id
	GROUP BY p.plan_name)
	SELECT
		*,
		ROUND(((customer_count_in_2020::numeric/total_customer_count::numeric)*100 ),2)as percentage
	FROM cte

(5 row(s) affected)

Total execution time: 00:00:00.004

plan_name,total_customer_count,customer_count_in_2020,percentage
pro annual,258,195,75.58
trial,1000,1000,100.00
churn,307,236,76.87
pro monthly,539,479,88.87
basic monthly,546,538,98.53


### <span style="font-size: 14px;"><b>8. How many customers have upgraded to an annual plan in 2020?</b></span>

<span style="font-size: 14px;">-- assumptions:</span>

<span style="font-size: 14px;"><span style="white-space:pre"></span>-- upgraded from any plan to pro annual plan</span>

<span style="font-size: 14px;">-- where year is 2020</span>

<span style="font-size: 14px;">-- use LEAD() to get next plan</span>

<span style="font-size: 14px;">-- count the customers</span>

In [9]:
with cte as(
	SELECT 
		s.customer_id,
		s.start_date,
		p.plan_name,
		LEAD(p.plan_name,1) OVER(partition by s.customer_id order by s.start_date) as next_plan
	FROM subscriptions s
	JOIN plans p
		ON s.plan_id = p.plan_id
		WHERE EXTRACT(YEAR FROM s.start_date) =2020)
	SELECT
		COUNT(*) as no_of_customers
		FROM cte
	WHERE next_plan='pro annual';

(1 row(s) affected)

Total execution time: 00:00:00.006

no_of_customers
195


### <span style="font-size: 14px;"><b>9. How many days on average does it take for a customer to an annual plan from the day they join Foodie-Fi?</b></span>

In [10]:
with cte2 as(
	with cte1 as(
		SELECT 
			s.customer_id,
			s.start_date,
			p.plan_name,
			LEAD(s.start_date,1) OVER(partition by s.customer_id order by s.start_date) as annual_plan_date,
			LEAD(p.plan_name,1) OVER(partition by s.customer_id order by s.start_date) as next_plan
		FROM subscriptions s
		JOIN plans p
			ON s.plan_id = p.plan_id
		WHERE p.plan_name = 'trial' or p.plan_name='pro annual')
	SELECT
		*,
		DATE_PART('day', annual_plan_date::timestamp-start_date::timestamp) as days_took_for_annual_plan
	FROM cte1
	where plan_name='trial' and next_plan='pro annual')
	SELECT
		avg(days_took_for_annual_plan) as avg_days_took_for_annual_plan
	FROM cte2

(1 row(s) affected)

Total execution time: 00:00:00.004

avg_days_took_for_annual_plan
104.62015503875969


### <span style="font-size: 14px;"><b>10.Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)</b></span>

In [11]:
with cte3 as(
	with cte2 as(
		with cte1 as(
			SELECT 
				s.customer_id,
				s.start_date,
				p.plan_name,
				LEAD(s.start_date,1) OVER(partition by s.customer_id order by s.start_date) as annual_plan_date,
				LEAD(p.plan_name,1) OVER(partition by s.customer_id order by s.start_date) as next_plan
			FROM subscriptions s
			JOIN plans p
				ON s.plan_id = p.plan_id
			WHERE p.plan_name = 'trial' or p.plan_name='pro annual')
		SELECT
			*,
			DATE_PART('day', annual_plan_date::timestamp-start_date::timestamp) as days_took_for_annual_plan
		FROM cte1
		where plan_name='trial' and next_plan='pro annual')
	SELECT
		customer_id,
		CASE
			WHEN days_took_for_annual_plan <=30 THEN '0-30 days'
			WHEN days_took_for_annual_plan <=60 THEN '31-60 days'
			WHEN days_took_for_annual_plan <=90 THEN '61-90 days'
			WHEN days_took_for_annual_plan <=120 THEN '91-120 days'
			WHEN days_took_for_annual_plan <=120 THEN '91-120 days'
			WHEN days_took_for_annual_plan <=150 THEN '121-150 days'
			WHEN days_took_for_annual_plan <=180 THEN '151-180 days'
			WHEN days_took_for_annual_plan <=210 THEN '181-210 days'
			WHEN days_took_for_annual_plan <=240 THEN '211-240 days'
			WHEN days_took_for_annual_plan <=270 THEN '241-270 days'
			WHEN days_took_for_annual_plan <=300 THEN '271-300 days'
			WHEN days_took_for_annual_plan <=330 THEN '301-330 days'
			WHEN days_took_for_annual_plan <=360 THEN '331-360 days'
		END as thirty_day_periods,
		days_took_for_annual_plan
	FROM cte2)
	SELECT
		thirty_day_periods,
		ROUND(avg(days_took_for_annual_plan)) as avg_days
	FROM cte3
	GROUP BY thirty_day_periods
	ORDER BY avg_days

(12 row(s) affected)

Total execution time: 00:00:00.005

thirty_day_periods,avg_days
0-30 days,10.0
31-60 days,42.0
61-90 days,71.0
91-120 days,101.0
121-150 days,133.0
151-180 days,162.0
181-210 days,191.0
211-240 days,224.0
241-270 days,257.0
271-300 days,285.0


### <span style="font-size: 14px;"><b>11. How many customers downgraded from a pro monthly to a basic monthly plan in 2020?</b></span>

<span style="font-size: 14px;">-- actual pro monthly to basic monthly not free trail pro monthly to basic monthly</span>

In [12]:
with cte as(
	SELECT 
		s.customer_id,
		s.start_date,
		p.plan_name,
		LEAD(p.plan_name,1) OVER(partition by s.customer_id order by s.start_date) as next_plan
	FROM subscriptions s
	JOIN plans p
		ON s.plan_id = p.plan_id
	WHERE EXTRACT(YEAR FROM s.start_date) =2020)
	SELECT
		COUNT(*)as no_of_customers
		FROM cte
	WHERE plan_name='pro monthly' and next_plan='basic monthly'

(1 row(s) affected)

Total execution time: 00:00:00.006

no_of_customers
0
